In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import seaborn as sns
import joblib

In [2]:
check = pd.read_csv('train.csv')

In [3]:
# remove the columns full of null values
data = check.dropna(axis=1)

In [4]:
data.isnull().sum()

Unnamed: 0                           0
id                                   0
Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Gate location                        0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
Departure Delay in Minutes           0
satisfaction                         0
dtype: int64

In [5]:
df_encoded = pd.get_dummies(data, columns=['Gender', 'Customer Type', 'Type of Travel', 'Class'], drop_first=True)

In [6]:
df_encoded['satisfaction'] = df_encoded['satisfaction'].map({
    'satisfied':1,
    'neutral or dissatisfied':0
})
df_encoded['Gender_Male'] = df_encoded['Gender_Male'].astype(int)
df_encoded['Customer Type_disloyal Customer'] = df_encoded['Customer Type_disloyal Customer'].astype(int)
df_encoded['Type of Travel_Personal Travel'] = df_encoded['Type of Travel_Personal Travel'].astype(int)
df_encoded['Class_Eco'] = df_encoded['Class_Eco'].astype(int)
df_encoded['Class_Eco Plus'] = df_encoded['Class_Eco Plus'].astype(int)

In [7]:
from xgboost import XGBClassifier

In [8]:
from sklearn.preprocessing import PolynomialFeatures
# Separate features and target variable
X = df_encoded[['Ease of Online booking','Gate location','Food and drink','Leg room service','Baggage handling','Class_Eco']]
y = df_encoded['satisfaction']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)

# Fit and transform the training data
X_train_poly = poly.fit_transform(X_train)

# Transform the test data
X_test_poly = poly.transform(X_test)

# Initialize and train the XGBClassifier
model = XGBClassifier(max_depth=10, n_estimators=200)
model.fit(X_train_poly, y_train)

# Save the model and the polynomial feature transformer
# poly_f = joblib.dump(model,'Model_XGB.pkl')
print("Model and transformer have been saved.")


Model and transformer have been saved.


In [9]:
load = joblib.load('Poly_XGB.pkl')

In [10]:
features = [3,	1,	4,	3,	2,	0]

In [11]:
features_poly = poly.transform(np.array(features).reshape(1, -1))
prediction = load.predict(features_poly)
print({'prediction': prediction[0]})

{'prediction': 0}


C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
